In [ ]:
%matplotlib notebook
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL
from IPython.display import Image, display

# These images are recorded by robot_io/realsense2_cam.py

image_dir = "/home/argusm/lang/gym_grasping/gym_grasping/flow_control/nissan_images"
keyframe = image_dir+"/20190808_112922.jpg"
keyframe_paint = keyframe.replace(".jpg","_paint.jpg")
seq_end =  image_dir+"/20190808_113821.jpg"

# compute flow between demo and live
#size = (640, 360)
size = (320, 180)

files = sorted(os.listdir(image_dir))
files = [os.path.join(image_dir, fn) for fn in files]
assert(keyframe in files and seq_end in files)

def load_image(fn):
    return PIL.Image.open(fn).resize(size).rotate(180)

image = load_image(keyframe_paint)
image = np.array(image)
mask = np.all(image == 255,axis=2)

In [ ]:
display(Image(filename=keyframe))

#display(Image(filename=keyframe_paint))
plt.imshow(mask)
plt.show()

In [ ]:
sequence = [fn for fn in files if (fn >= keyframe and fn <= seq_end and fn != keyframe_paint)]
print("Sequence length: ", len(sequence))

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np

image_sequence = [load_image(fn) for fn in sequence]

#print(image_sequence[0].size)
#print([64*i for i in range(25)])

fig, ax = plt.subplots(1,1)
line = ax.imshow(image_sequence[0])
ax.set_axis_off()

def update(i):
    image = image_sequence[i].copy()
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=len(image_sequence)-1,step=1,value=0,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)




In [ ]:
from gym_grasping.flow_control.flow_module import FlowModule
print(size)
flow_module = FlowModule(size=size)

In [ ]:
import io
import IPython

base_image = image_sequence[0]

def update(i):
    image = image_sequence[i].copy()
    flow = flow_module.step(np.array(base_image), np.array(image))
    
    flow[mask] = (0,0)
    print(np.mean(flow,axis=(0,1)))
    
    flow_img = flow_module.computeImg(flow, dynamic_range=False)
    
    with io.BytesIO() as output:
        PIL.Image.fromarray(flow_img).save(output, format="png")
        contents = output.getvalue()
        display( image, IPython.display.Image(data=output.getvalue()))

    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=len(image_sequence)-1,step=1,value=0,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)